# Multi-channel polarization

It is also possible to program simple methods, such as polarisation or current-voltage curves with Python itself in individual steps, instead of using methods from Zahner.

This slows down the sampling rate you can achieve, but has the advantage that you can write the data directly to your own file formats.

In [29]:
import sys
import math
import cmath
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode, ThalesRemoteScriptWrapper
import datetime
import time
import csv
import re

TARGET_HOST = "10.10.255.211"

zenniumConnection = ThalesRemoteConnection()
zenniumConnection.connectToTerm(TARGET_HOST)

zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
zahnerZennium.forceThalesIntoRemoteScript()

'128,ScriptRemote,5,6,0,0'

# Reading the ACQ setup

PAD4 ACQ channels could also be read DC, but for this they must be configured in the GUI and the ACQ setup must be saved. The setting of the PAD4 ACQ channels is explained in the PAD4 manual in chapter 6: https://doc.zahner.de/hardware/pad4.pdf

The ACQ setup can be read with [readAcqSetup()](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper.readAcqSetup).

The configured channel names for the file are then read from the setup.

In [30]:
print("acq configuration")

acq_setup = zahnerZennium.readAcqSetup()
print(f"setup string from the device:{acq_setup}")

index_channel_name_regex = r"DISP(?P<index>[\d+]);[\d+];(?P<channel_name>.*?);"
headers_match = re.finditer(index_channel_name_regex, acq_setup)

acq_channels = dict()
for match in headers_match:
    acq_channels[int(match.group("index"))] = match.group("channel_name")

print("found channels:")
for index, name in acq_channels.items():
    print(f"index:{index} channel name:{name}")

acq configuration
setup string from the device:OK;ACQSETUP;INPUTS;4;DISP0;2;PAD4Ch1;DISP1;3;PAD4Ch2;DISP2;4;PAD4Ch3;DISP3;5;PAD4Ch4;OUTPUTS;0;DACS;NONE;ACTIVE CHANNEL=0;ENDSETUP
found channels:
index:0 channel name:PAD4Ch1
index:1 channel name:PAD4Ch2
index:2 channel name:PAD4Ch3
index:3 channel name:PAD4Ch4


# Execute the constant current polarisation measurement

Now the current voltage and the ACQ channels are read out for a specified time at a specified sampling rate.

The number of ACQ channels determines the sampling frequency. In this example, reading out the 6 channels takes about 0.65 s.

In [31]:
zahnerZennium.calibrateOffsets()
zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_GALVANOSTATIC)
zahnerZennium.setCurrent(0.1)
zahnerZennium.enablePotentiostat()

duration = 10.0
sampling_frequency = 1.0

interval = 1.0 / sampling_frequency
samples = int(duration * sampling_frequency)
measurements = []
start_time = datetime.datetime.now()

for sample_index in range(samples):
    print(f"sample:{sample_index}/{samples}")
    loop_start = time.time()

    # measure all channels
    current_time = datetime.datetime.now()
    voltage = zahnerZennium.getPotential()
    current = zahnerZennium.getCurrent()
    acq_data = zahnerZennium.readAllAcqChannels()

    # store the results
    measurement = {
        "time": current_time,
        "voltage": voltage,
        "current": current,
        "index": sample_index,
    }

    for key, value in acq_data.items():
        measurement[key] = value

    measurements.append(measurement)

    # calculate the waiting time
    elapsed = time.time() - loop_start
    sleep_time = interval - elapsed

    # wait until the next measurement point
    if sleep_time > 0:
        time.sleep(sleep_time)
    else:
        print("sampling_frequency too high")

sample:0/10
sample:1/10
sample:2/10
sample:3/10
sample:4/10
sample:5/10
sample:6/10
sample:7/10
sample:8/10
sample:9/10


# Saving the data in a file

After the measurement, the data is written to a file. This could also be done as an alternative during the measurement as an example.
The channel designations that were read at the beginning of the example are now entered in the file as a header.

In [32]:
with open("polarization.txt", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter="\t")

    headers = ["time", "voltage", "current"]
    headers.extend([f"{value}" for key, value in acq_channels.items()])
    headers.append("index")

    writer.writerow(headers)

    for m in measurements:
        row = [m["time"].strftime("%Y-%m-%d %H:%M:%S.%f"), m["voltage"], m["current"]]
        for key, value in acq_channels.items():
            row.append(m[key])
        row.append(m["index"])

        writer.writerow(row)


zahnerZennium.disablePotentiostat()
zenniumConnection.disconnectFromTerm()
print("finish")

finish
